# ДЗ 2.1 
Шергалис Донат, 3530903/70301
Вариант 39.

In [1]:
from SPARQLWrapper import SPARQLWrapper, JSON
import requests
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import itertools
from operator import itemgetter 
import json


%matplotlib inline

In [2]:
is_test = False
# is_test = True

data_folder = 'test-data' if is_test else 'data'
var = 1 if is_test else 22

In [3]:
data = pd.read_csv(data_folder + '/data.csv', index_col=0).astype(float)
data.head()

,Movie 1,Movie 2,Movie 3,Movie 4,Movie 5,Movie 6,Movie 7,Movie 8,Movie 9,Movie 10,...,Movie 21,Movie 22,Movie 23,Movie 24,Movie 25,Movie 26,Movie 27,Movie 28,Movie 29,Movie 30
User 1,-1.0,4.0,1.0,5.0,-1.0,-1.0,-1.0,1.0,3.0,-1.0,...,2.0,4.0,-1.0,-1.0,3.0,4.0,-1.0,2.0,4.0,1.0
User 2,4.0,5.0,2.0,3.0,3.0,4.0,-1.0,5.0,3.0,2.0,...,-1.0,2.0,-1.0,3.0,1.0,5.0,-1.0,5.0,2.0,2.0
User 3,4.0,2.0,5.0,4.0,1.0,5.0,-1.0,3.0,5.0,2.0,...,4.0,-1.0,5.0,4.0,5.0,1.0,2.0,1.0,-1.0,3.0
User 4,5.0,-1.0,2.0,-1.0,-1.0,2.0,4.0,3.0,2.0,3.0,...,-1.0,2.0,2.0,2.0,2.0,5.0,3.0,2.0,5.0,5.0
User 5,4.0,3.0,3.0,3.0,5.0,3.0,3.0,4.0,5.0,2.0,...,4.0,5.0,2.0,-1.0,3.0,5.0,1.0,-1.0,5.0,-1.0


In [4]:
movie_names = pd.read_csv(data_folder + '/movie_names.csv', index_col=0, names=['id', 'name'], squeeze = True)
movie_names

id
Movie 1                              The Shawshank Redemption
Movie 2                                         The Godfather
Movie 3                                The Godfather: Part II
Movie 4                                       The Dark Knight
Movie 5                                          12 Angry Men
Movie 6                                      Schindler's List
Movie 7         The Lord of the Rings: The Return of the King
Movie 8                                          Pulp Fiction
Movie 9                         The Good the Bad and the Ugly
Movie 10                                           Fight Club
Movie 11     The Lord of the Rings: The Fellowship of the ...
Movie 12                                         Forrest Gump
Movie 13       Star Wars: Episode V - The Empire Strikes Back
Movie 14                                            Inception
Movie 15                The Lord of the Rings: The Two Towers
Movie 16                      One Flew Over the Cuckoo's Nest
Movie

In [5]:
weekdays = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']
context_day = pd.read_csv(data_folder + '/context_day.csv', index_col=0)
context_day.head()

,Movie 1,Movie 2,Movie 3,Movie 4,Movie 5,Movie 6,Movie 7,Movie 8,Movie 9,Movie 10,...,Movie 21,Movie 22,Movie 23,Movie 24,Movie 25,Movie 26,Movie 27,Movie 28,Movie 29,Movie 30
User 1,-1,Thu,Tue,Mon,-1,-1,-1,Tue,Sun,-1,...,Thu,Thu,-1,-1,Sun,Sat,-1,Wed,Wed,Thu
User 2,Mon,Wed,Mon,Sat,Sat,Thu,-1,Mon,Fri,Thu,...,-1,Sun,-1,Mon,Mon,Fri,-1,Sun,Tue,Wed
User 3,Tue,Wed,Sat,Tue,Sat,Mon,-1,Thu,Wed,Thu,...,Tue,-1,Wed,Thu,Tue,Thu,Wed,Wed,-1,Sun
User 4,Mon,-1,Wed,-1,-1,Sun,Sat,Mon,Mon,Wed,...,-1,Sun,Fri,Sat,Thu,Sat,Fri,Fri,Sat,Sun
User 5,Thu,Fri,Wed,Fri,Mon,Thu,Sat,Tue,Thu,Fri,...,Fri,Fri,Mon,-1,Wed,Mon,Sat,-1,Wed,-1


In [6]:
# h = дома
# c = кинотеатр
# v = в гостях
context_place = pd.read_csv(data_folder + '/context_place.csv', index_col=0)
context_place.head()

,Movie 1,Movie 2,Movie 3,Movie 4,Movie 5,Movie 6,Movie 7,Movie 8,Movie 9,Movie 10,...,Movie 21,Movie 22,Movie 23,Movie 24,Movie 25,Movie 26,Movie 27,Movie 28,Movie 29,Movie 30
,,,,,,,,,,,,,,,,,,,,,
User 1,-1,h,h,c,-1,-1,-1,v,h,-1,...,h,c,-1,-1,c,c,-1,v,h,c
User 2,v,v,v,v,h,c,-1,v,h,c,...,-1,c,-1,c,h,v,-1,v,v,c
User 3,v,h,c,c,c,c,-1,h,c,v,...,v,-1,v,v,h,h,h,v,-1,v
User 4,v,-1,h,-1,-1,h,v,v,c,h,...,-1,v,h,h,c,v,c,v,v,c
User 5,c,c,c,c,v,h,v,v,c,h,...,v,c,v,-1,h,c,v,-1,h,-1


In [7]:
k = 4
round_to = 3

films_count = len(data.T)
films_names = data.columns

users_count = len(data)
users_names = data.T.columns

user_name = users_names[var - 1]
# assert user_name == 'User 39'

# Получить строчку для нашего пользователя
def for_user(u_name = user_name, data = data):
    return data.T[u_name]

## Задание 1. 

Для заданного пользователя (совпадает с вашим номером в таблице) рассчитать оценки для всех фильмов, которые он не оценил.

В качестве метрики сходства использовать метрику косинуса:  
$ sim_{u,v} = \frac { \sum_i{u_i v_i} } { \sum_i{u_i^2} * \sum_i{v_i^2} }  $

$ u_i $ - оценка пользователя u для фильма i,  
$ u_i $ - оценка пользователя v для фильма i  

In [8]:
# sim для пары пользователей
def get_sim(u_name, v_name = user_name, data = data):
    u = for_user(u_name, data)
    v = for_user(v_name, data)
    return calc_sim(u, v)

# sim для пары массивов
def calc_sim(u, v):
    sum_top = 0
    sum_bottom_first = 0
    sum_bottom_second = 0
    
    for i in range(len(u)):
        if u[i] > 0 and v[i] > 0:
            sum_top += u[i] * v[i]
            sum_bottom_first += u[i] ** 2
            sum_bottom_second += v[i] ** 2

    if (sum_top == 0): return 0
    return round(sum_top / (np.sqrt(sum_bottom_first) * np.sqrt(sum_bottom_second)), round_to)

In [9]:
# test calc_sim
a = [3, 5, -1,  5]
b = [3, 4,  5, -1]
expected = ((3 * 3) + (5 * 4)) / (((3**2 + 5**2)**0.5) * ((3**2 + 4**2)**0.5))
actual = calc_sim(a, b)
assert expected - actual < 1e-09
assert 1 - calc_sim(a, a) < 1e-09
# /test

In [10]:
sim = {}
for u_name in users_names:
    sim[u_name] = get_sim(u_name)

sorted_sim = sorted(sim.items(), key = itemgetter(1), reverse = True)[1:]
# Самые похожие пользователи:
print(sorted_sim[:4])
closest_users = [name for name, sim in sorted_sim[:4]]

[('User 12', 0.932), ('User 26', 0.93), ('User 38', 0.924), ('User 5', 0.917)]


Рассчёт оценки $ r_{ui} $ пользователя $ u $ для фильма $ i $ :  
$ r_{ui} = r_u + \frac{ \sum_v^k{sim_{vu}} \cdot (r_{vi} - r_v) }{ \sum_v^k {|sim_{vu}|} } $  

$ r $ - рассчитываемая оценка,  
$ r_u $ - средняя оценка у пользователя u,  
$ r_v $ - средняя оценка у пользователя v,   
$ r_{vi} $ - оценка пользователя v для фильма i,  
$ sim_{vu} $ - значение метрики сходства для пользователей u и v

In [11]:
def get_rate(film_name, u_name = user_name, data = data):
    return for_user(u_name, data)[film_name]

# 0..k имён пользователей с оценками для film_name с наибольшими sim
def get_k_closest_for_film(film_name, closest_users = closest_users, k = k, data = data):
    return [u_name for u_name in closest_users if get_rate(film_name, u_name, data) != -1]

In [12]:
# test get_k_closest_for_film 
c = ['u1', 'u2']
d = pd.DataFrame.from_dict({ 'm1': { 'u1':-1, 'u2':3 }, 'm2': { 'u1':5, 'u2':-1 } })
expected = ['u1']
actual = get_k_closest_for_film('m2', c, 2, d)
assert expected == actual
# /test get_k_closest_for_film

In [13]:
# получить среднюю оценку пользователя
def get_avg(u_name = user_name, data = data):
    all_rates = for_user(u_name, data)
    existing_rates = [rate for rate in all_rates if rate != -1]
    return np.average(existing_rates)

In [14]:
# test get_avg
d = pd.DataFrame.from_dict({ 'm1': { 'u1':-1, 'u2':3 }, 'm2': { 'u1':5, 'u2':-1 } })
expected = 3
actual = get_avg('u2', d)
assert expected == actual
# /test get_avg

In [15]:
# r_ui пользователя u и фильма i
def calc_rate(film_name, u_name = user_name, sim = sim, data = data):
    u_avg = get_avg(u_name, data)
    
    sum_top = 0
    closest_for_film = get_k_closest_for_film(film_name)
    for v_name in closest_for_film:
        r_vi = for_user(v_name)[film_name]
        if v_name != u_name and r_vi != -1:
            r_v = get_avg(v_name)
            sum_top += sim[v_name] * (r_vi - r_v)
    
    sum_bottom = np.sum([sim[v_name] for v_name in closest_for_film if v_name != u_name])
    
    return round(u_avg + sum_top / sum_bottom, round_to)

In [16]:
user_rates = for_user()
predicted_rates = {}
for film_name in films_names:
    if user_rates[film_name] < 0:
        predicted_rates[film_name] = round(calc_rate(film_name), round_to)

In [17]:
for film_name, rating in predicted_rates.items():
    print(f"{film_name}\tpredicted: {predicted_rates[film_name]}\t{movie_names[film_name.strip()]}")

 Movie 4	predicted: 3.476	 The Dark Knight
 Movie 6	predicted: 2.807	 Schindler's List
 Movie 17	predicted: 1.448	 Goodfellas
 Movie 18	predicted: 3.701	 The Matrix
 Movie 24	predicted: 4.834	 It's a Wonderful Life
 Movie 25	predicted: 2.198	 Life Is Beautiful
 Movie 30	predicted: 4.295	 The Green Mile


## Задание 2. 
Порекомендовать заданному пользователю 1 фильм, который он посмотрел бы в выходной дома, если такой фильм есть в данных.

### Описание алгоритма
Для этого для каждого не посмотренного фильма нашего пользователя посчитаем некий вес рекомендации другого пользователя knn:  
* 1, если совпали с требуемыми время И место просмотра  
* t, если совпали время ИЛИ место просмотра  
* 0, если смотрел в другое время в другом месте
* -1, если пользователь не смотрел этот фильм

Для каждого фильма посчитаем среднее между всеми неотрицательными весами и выберем фильм с наибольшим ненулевым средним. Если такого не окажется, то рекомендации не будет.

Можно игнорировать "нестрогие" совпадения, установив для t значение -1.  
Можно считать "нестрогие" совпадения несовпадениями, установив для t значение 0.

### Описание алгоритма более человеческим языком:  
Возьмём "друзей" нашего пользователя. Начнём перебирать фильмы -- если "друг" смотрел фильм:
* в нужное время в нужном месте, добавим 1
* в нужное время ИЛИ в нужном месте, добавим t (если t >= 0)
* в другое время И в другом месте, добавим 0
* не смотрел совсем, ничего не добавим

Посчитаем среднее, выберем фильм с максимальным ненулевым средним.
Если такого нет, то рекомендации не будет.

В зависимости от того, насколько релевантными мы считаем совпадения (время ИЛИ места), можем менять значение t от 0 до 1, или установить значение -1, чтобы их игнорировать вовсе.

In [18]:
watch_where = ['h']
watch_when = weekdays[5:]
print('Searching films to watch at ', watch_where, " on ", watch_when)

t = 0.3

Searching films to watch at  ['h']  on  ['Sat', 'Sun']


In [19]:
# верни 1, если совпали день И место
# верни t, если совпал день ИЛИ место
def get_context_weight(film_name, user_name, watch_where = watch_where, watch_when = watch_when, t = t, context_place = context_place, context_day = context_day):
    place_match = for_user(user_name, context_place)[film_name].strip() in watch_where
    day_match = for_user(user_name, context_day)[film_name].strip() in watch_when
    return 1 if place_match and day_match \
        else t if place_match or day_match \
        else 0

In [20]:
films_weights = {}

for film_name, predicted_rate in predicted_rates.items():
    get_context_weight_for_this = lambda user_name :\
                                    get_context_weight(film_name, user_name, watch_where, watch_when)
    film_rates = data[film_name]
    films_weights[film_name] = []
    for friend_name in closest_users:
        if film_rates[friend_name] != 1:
            weight = film_rates[friend_name] * sim[friend_name] * get_context_weight_for_this(friend_name)
            
            if weight >= 0: films_weights[film_name].append(round(weight, round_to))
            
films_weights

{' Movie 4': [-0.0, 0.0, 3.696, 0.0],
 ' Movie 6': [-0.0, 0.558, 0.832, 0.825],
 ' Movie 17': [0.0, 0.55],
 ' Movie 18': [1.398, 0.837, 1.109, 0.0],
 ' Movie 24': [1.398, 4.65, -0.0, -0.0],
 ' Movie 25': [0.0, 0.558, 0.825],
 ' Movie 30': [0.0, -0.0, 1.109, -0.0]}

In [21]:
films_weights_total = {film_name: round(np.average(weights), round_to) for film_name, weights in films_weights.items()}
films_weights_total

{' Movie 4': 0.924,
 ' Movie 6': 0.554,
 ' Movie 17': 0.275,
 ' Movie 18': 0.836,
 ' Movie 24': 1.512,
 ' Movie 25': 0.461,
 ' Movie 30': 0.277}

In [22]:

print('Recomendation for', user_name, watch_where, watch_when, ':')
if (max(films_weights_total.values()) > 0):
    recomendation = max(films_weights_total, key=films_weights_total.get)
    recomendation_name = movie_names[recomendation.strip()].strip()
    
    print(f'\t {recomendation} ({recomendation_name})')
else: 
    recomendation = None
    print('\t', None)

Recomendation for User 22 ['h'] ['Sat', 'Sun'] :
	  Movie 24 (It's a Wonderful Life)


## Вывод 

In [23]:
predicted_rates_striped = { film.strip(): rate for film, rate in predicted_rates.items() }

In [24]:
output = {
    "user": var,
    "1": predicted_rates_striped,
    "2": { recomendation.strip(): predicted_rates[recomendation]} if recomendation else None
}

output_json=json.dumps(output, indent = 2)
print(output_json)

{
  "user": 22,
  "1": {
    "Movie 4": 3.476,
    "Movie 6": 2.807,
    "Movie 17": 1.448,
    "Movie 18": 3.701,
    "Movie 24": 4.834,
    "Movie 25": 2.198,
    "Movie 30": 4.295
  },
  "2": {
    "Movie 24": 4.834
  }
}


In [25]:
with open('shergalis_2.1.json', 'w') as outfile:
    outfile.write(output_json)

# 3.2 SPARQL-запрос


In [26]:
film_names = [movie_names[film_name.strip()].strip() for film_name in predicted_rates.keys()]
film_names

['The Dark Knight',
 "Schindler's List",
 'Goodfellas',
 'The Matrix',
 "It's a Wonderful Life",
 'Life Is Beautiful',
 'The Green Mile']

In [27]:
api = "https://www.wikidata.org/w/api.php"

def get_wd_by_name(name):
    params = {
        'action' : 'wbsearchentities',
        'format' : 'json',
        'language' : 'en',
        'search': name
    }
    res = requests.get(api, params = params)
    
    for res in res.json()['search']:
        if "film" in res['description']:
            return res['id']

    raise f"Film {name} not found!" + res.json()

In [28]:
film_wd = {name:get_wd_by_name(name) for name in film_names}
film_wd

{'The Dark Knight': 'Q163872',
 "Schindler's List": 'Q483941',
 'Goodfellas': 'Q42047',
 'The Matrix': 'Q83495',
 "It's a Wonderful Life": 'Q204191',
 'Life Is Beautiful': 'Q19355',
 'The Green Mile': 'Q208263'}

In [29]:
sparql = SPARQLWrapper("https://query.wikidata.org/sparql")

query_template = """
# query for film: '$FILM_NAME$'
SELECT ?title ?screenwriterLabel ?screenwriterEducationLabel ?screenwriterEducationLocation
WHERE {
         
  wd:$FILM_WD$ wdt:P1476 ?title;
               wdt:P58 ?screenwriter.
  ?screenwriter wdt:P69 ?screenwriterEducation.
  ?screenwriterEducation wdt:P625 ?screenwriterEducationLocation.

  SERVICE wikibase:label { bd:serviceParam wikibase:language "ru,en". }
}
"""

In [30]:
queries = { name:query_template
           .replace("$FILM_WD$", wd)
           .replace("$FILM_NAME$", name) 
           for name, wd in film_wd.items() }

In [31]:
def execute_query(query):
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    return sparql.query().convert()

In [32]:
queries_results = {name:execute_query(query) for name, query in queries.items()}

In [33]:
pd_results = {name:pd.json_normalize(result['results']['bindings']) for name, result in queries_results.items()}

In [34]:
sparql_results =  {name:result[['screenwriterLabel.value', 
                                'screenwriterEducationLabel.value', 
                                'screenwriterEducationLocation.value']] 
                   for name, result in pd_results.items()}

In [35]:
sparql_results_list = list(sparql_results.items())
def show(id):
    print(sparql_results_list[id][0])
    return sparql_results_list[id][1]

In [36]:
show(0)

The Dark Knight


,screenwriterLabel.value,screenwriterEducationLabel.value,screenwriterEducationLocation.value
0,Дэвид Гойер,Университет Южной Калифорнии,Point(-118.285278 34.021667)
1,"Кейн, Боб",Купер-юнион,Point(-73.99058 40.72927)
2,Кристофер Нолан,Университетский колледж Лондона,Point(-0.1340401 51.5245592)
3,Джонатан Нолан,Джорджтаунский университет,Point(-77.072777777 38.907222222)
4,Дэвид Гойер,Школа кинематографических искусств Университет...,Point(-118.285833 34.023056)
5,"Кейн, Боб",Средняя школа ДеВитта Клинтона,Point(-73.8865 40.8811)
6,Билл Фингер,Средняя школа ДеВитта Клинтона,Point(-73.8865 40.8811)
7,Кристофер Нолан,Haileybury and Imperial Service College,Point(-0.033333 51.7787)


In [37]:
show(1)

Schindler's List


,screenwriterLabel.value,screenwriterEducationLabel.value,screenwriterEducationLocation.value
0,Стивен Заиллян,Университет штата в Сан-Франциско,Point(-122.479722222 37.723333333)


In [38]:
show(2)

Goodfellas


,screenwriterLabel.value,screenwriterEducationLabel.value,screenwriterEducationLocation.value
0,Мартин Скорсезе,Тишская школа искусств,Point(-73.99377778 40.72922222)


In [39]:
show(3)

The Matrix


,screenwriterLabel.value,screenwriterEducationLabel.value,screenwriterEducationLocation.value
0,Лили Вачовски,Колледж Эмерсон,Point(-71.065994 42.351807)
1,Лили Вачовски,Whitney M. Young Magnet High School,Point(-87.6636 41.8782)


In [40]:
show(4)

It's a Wonderful Life


,screenwriterLabel.value,screenwriterEducationLabel.value,screenwriterEducationLocation.value
0,Фрэнсис Гудрич,Колумбийский университет,Point(-73.961944444 40.8075)
1,"Капра, Фрэнк",Калифорнийский технологический институт,Point(-118.125 34.1375)
2,Майкл Уилсон,Калифорнийский университет в Беркли,Point(-122.259 37.87)
3,Филип Ван Дорен Стерн,Ратгерский университет,Point(-74.448055555 40.501666666)
4,Фрэнсис Гудрич,Колледж Вассара,Point(-73.895188888 41.686866666)
5,Фрэнсис Гудрич,Columbia University School of Social Work,Point(-73.958352777 40.810252777)


In [41]:
show(5)

Life Is Beautiful


,screenwriterLabel.value,screenwriterEducationLabel.value,screenwriterEducationLocation.value
0,"Черами, Винченцо",Римский университет Ла Сапиенца,Point(12.515833333 41.903333333)


In [42]:
show(6)

The Green Mile


,screenwriterLabel.value,screenwriterEducationLabel.value,screenwriterEducationLocation.value
0,"Дарабонт, Фрэнк",Голливудская старшая школа,Point(-118.340068 34.099148)


In [43]:
show(7) # no

IndexError: list index out of range